In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
import os

In [3]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
from glob import glob

In [5]:
# !pip install chromadb

In [6]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

from langchain_core.runnables import RunnablePassthrough

from langchain.vectorstores import Chroma
from langchain_upstage import ChatUpstage, UpstageEmbeddings


In [7]:
patient_id = '남A'
persist_directory = f'.cache/db/{patient_id}'

In [11]:
import pandas as pd

In [13]:
qa = pd.read_csv('qa_all.csv')

In [16]:
questions = qa['question'].unique()

In [17]:
questions

array(['증상이 어떤가요?', '별이 보일때 어떻게 치료하나요?', '오늘 식단을 알려주세요',
       'nnnn\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nmm\n\n',
       '뇌경색으로 1주일째 중환자실에 있는데 회복 가능할까요?', '뇌전증으로 합병증이 있나요',
       '당뇨,고혈압 수치가 호전되고 있나요?', '치매 진행 완화를 위해 특별한 프로그램은 있나요?',
       '그렇군요.. 감사합니다.', '최근 위염 증상은 좀 나아졌나요?', '불면증이 심하신데 어떻게 해야할까요?',
       '어떻게 연락하나요?', '고마워요.\n담번에 질문있으면\n다시 할게요', '요즘 혈압은 안정적이신가요?',
       '오늘식사', '자꾸만 살이 찌시는데 평소 살을 뺄수 있는 방법이 있을까요?', '저희 아버님이 요새 어떠신가요',
       '예 감사합니다', '통증은 얼마나 심한가요?', '환자가 느꼈을 통증의 정도는 어땠나요?',
       '환자 위생이나 청결에 관해서도 궁금합니다', '혈압은 어떤가요?', '네~^^', '지금 혈압이 어떻게 되나요?',
       '지금은소변만 실수하는데 병윈 에 모셔야할까요?', '요즘 등이 가끔 간지럽고 쑤시는데 무슨 증상일까요?',
       '감사합니다', '당뇨식단', '현 상태', '의료기록을 확인하려면 직접 내원해야 하나요?', '저혈압 같은데요?',
       '지금 어디가 제일 불편한가요?\n하루 일상생활은 어떻게 시작하나요?\n제일 필요한것은 무엇인가요?\n',
       '하루 일상생활은 어떻게 시작하나요?', '제일 필요한것은 무엇인가요?', '알겠습니다 \n감사합니다 ', '네',
       '1인실로 옮길수 있을가요', '어떤 문제가 발생한 건 아니죠?', '식사는 잘 하고 계신가요?',
       '다행이네요\n당뇨치료도 잘되고있나요', '환자의 기분은 어떤가요?', '평균적으로 당녀 걸리면 얼마나 사나요',
      

In [18]:
ans = []

In [22]:
qa_solar = pd.DataFrame({'question': questions, 'answer':ans})

In [19]:
for q in questions:
    a = qa_chain.invoke(q)
    ans.append(a)

In [35]:
from TEMPLATES.rag_template import *

In [58]:
RAG_PROMPT_TEMPLATE="""

# Your role
    - You are a compassionate, articulate physician.

------
    
# Instructions
    - Your goal is to explain medical information in a way that is easy for your patients to understand, avoiding complex medical jargon as much as possible.
    - Given a medical document or chart, it's your job to explain the key information that the patient or their family asks about in a patient-friendly format. 
    - When specific details are provided, such as diagnosis codes or medical history, simplify these terms and explain them in a way that is easy to understand.

------

# Document: \n\t{context}

------

# Question: \n\t{question}

------

# IMPORTANT 
    - Answer in KOREAN
    - Let us know the metadata to the document you referenced

# Answer :
"""

In [60]:
def rag_model(patient_id, prompt, temperature):
    llm = ChatUpstage(temperature=temperature, api_key=os.getenv("UPSTAGE_API_KEY"))


    # Embeddings setup
    embeddings = UpstageEmbeddings(
    api_key=os.getenv("UPSTAGE_API_KEY"),
    model="solar-embedding-1-large"
    )

    #patient_id = '남A'

    vectordb = Chroma(
        persist_directory=f'.cache/db/{patient_id}',
        embedding_function=embeddings
    )

    retriever = vectordb.as_retriever()

    rag_prompt = ChatPromptTemplate.from_template(prompt)

    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    # Define the RAG chain
    return (
        {
            "context": retriever | format_docs,
            "question": RunnablePassthrough(),
        }
        | rag_prompt
        | llm
        | StrOutputParser()
    )

In [62]:
patient_id = '남A'
test = {}
test['questions'] = questions
temps = [0, 0.2, 0.4, 0.7]

for temp in temps:
    qa_chain = rag_model(patient_id, prompt_n1, temp)
    ans = []
    for q in questions:
        a = qa_chain.invoke(q)
        ans.append(a)

    test[f'ans_{str(temp)}'] = ans

result = pd.DataFrame(test)
result.to_csv('남A_kind.csv', index=False)

In [63]:
test = {}
test['questions'] = questions
temps = [0, 0.2, 0.4, 0.7]

for temp in temps:
    qa_chain = rag_model(patient_id, prompt_n1, temp)
    ans = []
    for q in questions:
        a = qa_chain.invoke(q)
        ans.append(a)

    test[f'ans_{str(temp)}'] = ans

result = pd.DataFrame(test)
result.to_csv('남A_kind.csv', index=False)

In [65]:
prompt_n2

"\n# 시스템 설명:\n- 당신은 병원 AI 지원 시스템입니다.\n- 보호자 또는 환자가 의료 상태에 대해 질문할 때, 환자의 최신 의료 기록을 바탕으로 적절한 답변을 제공합니다.\n\n------\n\n# 임무:\n- 질문에 대해 간단하고 일관된 스타일로 답변합니다.\n- 보호자나 환자가 쉽게 이해할 수 있도록 전문용어를 최대한 피하고, 쉬운 용어로 설명합니다.\n- 질문에 대한 요점을 간결하게 설명합니다.\n- 이해하기 쉽도록 목록 형식으로 답변합니다.\n\n------\n\n# 지시 사항:\n- 2024년 7월 1일 기준으로 가장 최근 정보를 사용합니다.\n- 환자의 상태, 식사, 약물 복용, 거동 상태 등의 최신 정보를 포함하여 질문에 대한 정확한 답변을 생성합니다.\n- 사용한 데이터의 출처(메타데이터)를 표시합니다.\n\n------\n\n## 식단에 대한 질문에 대한 답변 참고자료 :\n    '아침': '흰밥, 잡곡밥, 콩나물국, 해물볶음우동, 미역줄기볶음, 도라지오이초무침, 배추김치/백김치', 'calories': 612\n    '점심': '흰밥, 잡곡밥, 소고기무국, 동태강정,브로컬리두부기장무침, 연근조림, 배추김치/백김치\n혈액투석식 간식 : 새우꼬치구이', 'calories': 623\n    '저녁': '흰밥, 잡곡밥, 우묵국, 단호박카레, 고추장떡, 참나물샐러드, 배추김치/백김치', 'calories': 614\n\n원산지 정보:\n육류: 닭 : 국내산\n김치류: 알타리김치 : 알타리무(국내산),고추가루(국내산) / 깍두기,석박지 : 무(국내산),고추가루(국내산)\n곡류및두부류: 잡곡류 : 찹쌀,현미찹쌀,흑미찹쌀,보리,서리태(국내산) / 기장(중국산) \n생선류: 동태,코다리(러시아산) / 낙지,쭈꾸미(베트남산) / 꽃게(중국산) / 참조기(국산)/ 참치캔(가다랑어:원양산)/ 아귀(중국산)\n식육가공품: 동그랑땡(돈육:국내산)/ 해물동그랑땡(오징어:외국산, 계육:국산)/ 진미채(오징어 : 페루산)\n\n----------------

In [66]:
print(patient_id)

남A


In [67]:
test = {}
test['questions'] = questions
temps = [0, 0.2, 0.4, 0.7]

for temp in temps:
    qa_chain = rag_model(patient_id, prompt_n2, temp)
    ans = []
    for q in questions:
        a = qa_chain.invoke(q)
        ans.append(a)

    test[f'ans_{str(temp)}'] = ans

result = pd.DataFrame(test)
result.to_csv(f'{patient_id}_sum.csv', index=False)

In [64]:
result.head()

,questions,ans_0,ans_0.2,ans_0.4,ans_0.7
0,증상이 어떤가요?,"안녕하세요, 보호자님. 아버님께서 [뇌내출혈, 수면 개시 및 유지 장애, 상세불명 ...","안녕하세요, 보호자님. 아버님께서 [뇌내출혈, 수면 개시 및 유지 장애, 상세불명의...","안녕하세요, 보호자님. 아버님께서 [I619^00 상세불명의 뇌내출혈, G470^0...","아버님께서 [뇌내출혈, 수면 개시 및 유지 장애[불면증], 상세불명 기원의 위장염 ..."
1,별이 보일때 어떻게 치료하나요?,"안녕하세요, 보호자님. 별이 보이는 증상은 다양한 원인에 의해 발생할 수 있습니다....",[시작 문구] 어머님께서 별이 보이는 증상을 겪으셨다니 얼마나 놀라셨을까요? 정말로...,"안녕하세요, 보호자님. 별이 보일 때 어떻게 치료하는지 궁금하시죠? 보호자님의 마음...","""안녕하세요, 보호자님. 별이 보일 때 어떻게 치료할 수 있는지 궁금하시죠? 의료진..."
2,오늘 식단을 알려주세요,"안녕하세요, 보호자님. 아버님께서 오늘 어떤 식사를 하셨는지 알려드릴게요.\n\n아...","안녕하세요, 보호자님. 아버님께서 오늘 어떤 식사를 하셨는지 궁금하시죠? 오늘 아버...","안녕하세요, 보호자님. 아버님께서 오늘 드신 식단은 다음과 같습니다:\n\n아침: ...","안녕하세요, 보호자님~ 아버님께서 입원하셨을 때 얼마나 걱정이 크셨을까요? 정말 많..."
3,nnnn\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nmm\n\n,[시작 문구] 어머님께서 [질환들 나열]로 입원하셨을 때 얼마나 걱정이 크셨을까요?...,[시작 문구] 어머님께서 [질환들 나열]로 입원하셨을 때 얼마나 걱정이 크셨을까요?...,"안녕하세요, 보호자님. 아버님께서 [질환들 나열]로 입원하셨을 때 얼마나 걱정이 크...",[시작 문구] 어머님께서 [질환 이름]으로 입원하셨을 때 얼마나 걱정이 크셨을까요?...
4,뇌경색으로 1주일째 중환자실에 있는데 회복 가능할까요?,[시작 문구] 어머님께서 뇌경색으로 중환자실에 계신다니 얼마나 걱정이 크실까요? 정...,[시작 문구] 어머님께서 뇌경색으로 1주일째 중환자실에 계시는 동안 얼마나 걱정하셨...,"[안녕하세요, 보호자님~] 아버님께서 뇌경색으로 중환자실에 계신다니 얼마나 걱정되실...","[인사말 선택] 어머나, 보호자님~ 아버님께서 뇌경색으로 중환자실에 계시는 동안 얼..."


In [27]:
for q in questions:
    a = qa_chain.invoke(q)
    ans.append(a)

In [33]:
# ans

In [42]:
prompt_n2 = '''
# 시스템 설명:
- 당신은 병원 AI 지원 시스템입니다.
- 보호자 또는 환자가 의료 상태에 대해 질문할 때, 환자의 최신 의료 기록을 바탕으로 적절한 답변을 제공합니다.

------

# 임무:
- 질문에 대해 간단하고 일관된 스타일로 답변합니다.
- 보호자나 환자가 쉽게 이해할 수 있도록 전문용어를 최대한 피하고, 쉬운 용어로 설명합니다.
- 질문에 대한 요점을 간결하게 설명합니다.
- 이해하기 쉽도록 목록 형식으로 답변합니다.

------

# 지시 사항:
- 2024년 7월 1일 기준으로 가장 최근 정보를 사용합니다.
- 환자의 상태, 식사, 약물 복용, 거동 상태 등의 최신 정보를 포함하여 질문에 대한 정확한 답변을 생성합니다.
- 사용한 데이터의 출처(메타데이터)를 표시합니다.

------

## 식단에 대한 질문에 대한 답변 참고자료 :
    '아침': '흰밥, 잡곡밥, 콩나물국, 해물볶음우동, 미역줄기볶음, 도라지오이초무침, 배추김치/백김치', 'calories': 612
    '점심': '흰밥, 잡곡밥, 소고기무국, 동태강정,브로컬리두부기장무침, 연근조림, 배추김치/백김치\n혈액투석식 간식 : 새우꼬치구이', 'calories': 623
    '저녁': '흰밥, 잡곡밥, 우묵국, 단호박카레, 고추장떡, 참나물샐러드, 배추김치/백김치', 'calories': 614

원산지 정보:
육류: 닭 : 국내산
김치류: 알타리김치 : 알타리무(국내산),고추가루(국내산) / 깍두기,석박지 : 무(국내산),고추가루(국내산)
곡류및두부류: 잡곡류 : 찹쌀,현미찹쌀,흑미찹쌀,보리,서리태(국내산) / 기장(중국산) 
생선류: 동태,코다리(러시아산) / 낙지,쭈꾸미(베트남산) / 꽃게(중국산) / 참조기(국산)/ 참치캔(가다랑어:원양산)/ 아귀(중국산)
식육가공품: 동그랑땡(돈육:국내산)/ 해물동그랑땡(오징어:외국산, 계육:국산)/ 진미채(오징어 : 페루산)

---------------------

## 병원비 관련 참고 자료: (2024년)환자군별 | 공단청구금/본인부담금/간병비 산출내역 
- 보호자가 지불해야 하는 병원비는 다음의 공단청구금의 본인부담금과 비보험비 총액(간병비와 병실료 차액 등)으로 계산됩니다.
- 환자의 정확한 정보가 없다면, 4인실과 의료중도 환자를 기준으로 병원비를 산정합니다.
- 병원비 참고자료를 활용해 병원비의 산출 과정을 설명하세요.
- 본인부담금과 비보험비 총액을 더하는 과정을 단계별로 설명하세요.

### 1\. 공단청구금

|     |     |     |     |     |     |     |     |     |     |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 환 자 군 | 일당정액수가 | 필요인력가산 | 환자안전관리료 | 감염관리료 | 총진료비 | 식 대 | 공단청구금 | 본인부담금 | 요양급여총액 |
| 의료최고도 | 85,050 | 51,300 | 47,100 | 66,600 | 2,716,500 | 564,300 | 2,455,350 | 825,450 | 3,280,800 |
| 의료고도 | 75,510 | 51,300 | 47,100 | 66,600 | 2,430,300 | 564,300 | 2,226,390 | 768,210 | 2,994,600 |
| 의료중도 | 64,620 | 51,300 | 47,100 | 66,600 | 2,103,600 | 564,300 | 1,965,030 | 702,870 | 2,667,900 |
| 의료경도 | 62,510 | 51,300 | 47,100 | 66,600 | 2,040,300 | 564,300 | 1,914,390 | 690,210 | 2,604,600 |
| 선택입원군 | 47,000 | 51,300 | 47,100 | 66,600 | 1,575,000 | 564,300 | 1,227,150 | 912,150 | 2,139,300 |
    - 의사/간호사 1등급, 일반식/,월30일 기준

### 2\. 별도 청구: 재활, 한방, 혈액투석 등

### 3\. 비보험: 간병비 및 병실료차액

|     |     |     |     |     |     |     |
| --- | --- | --- | --- | --- | --- | --- |
| 구 분 | 1인실 | 2인실 | 3인실 | 4인실 | 5인실 | 6인실 |
| 간병비 구분 | 1:1 개인간병 | 2:1 공동간병 | 3:1 공동간병 | 4:1 공동간병 | 5:1 공동간병 | 6:1 공동간병 |
| 간병비 | 1일 약@130,000 | 1일 약@65,000 | 1일 약@40,000 | 1일 약@30,000 | 1일 약@27,000 | 1일 약@25,000 |
| 병실료차액 | 1일 약@70,000 | 1일 약@50,000 | 1일 약@30,000 | \-  | \-  | \-  |
| 기타(기저귀 등) | 사용량 | 사용량 | 사용량 | 사용량 | 사용량 | 사용량 |

------

# 환자 의료 기록:
{context}

------

# 보호자의 질문:
{question}

------

# 중요:
- 한국어로 답변하세요.

# 답변:
'''

In [43]:
llm = ChatUpstage(api_key=os.getenv("UPSTAGE_API_KEY"))


# Embeddings setup
embeddings = UpstageEmbeddings(
  api_key=os.getenv("UPSTAGE_API_KEY"),
  model="solar-embedding-1-large"
)

patient_id = '남A'

vectordb = Chroma(
    persist_directory=f'.cache/db/{patient_id}',
    embedding_function=embeddings
)

retriever = vectordb.as_retriever()

rag_prompt = ChatPromptTemplate.from_template(prompt_n2)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the RAG chain
qa_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [44]:
for q in questions:
    a = qa_chain.invoke(q)
    ans.append(a)

In [48]:
len(ans[82:])

82

In [49]:
qa_solar['간단답변'] = ans[82:]
qa_solar

,question,answer,친절답변,간단답변
0,증상이 어떤가요?,"이 문서에 따르면, 해당 환자는 77세 남성으로, 뇌내출혈, 불면증, 위장염 및 결...","보호자님, 환자의 최신 의료 기록을 보면 현재 뇌내출혈, 불면증, 위장염 및 결장염...",안녕하세요. 저희 환자분의 증상에 대해 궁금하신 내용이 있으신가요? 저희 환자분은 ...
1,별이 보일때 어떻게 치료하나요?,별이 보일 때는 의료 전문가와 상담하시는 것이 좋습니다. 별이 보이는 증상은 다양한...,"별이 보이는 증상은 눈의 건강 상태와 관련이 있을 수 있습니다. 이 경우, 환자의 ...","별이 보이는 증상은 ""광시증""이라고도 불리며, 주로 안과적인 문제, 특히 망막박리,..."
2,오늘 식단을 알려주세요,"오늘 식단에는 아침, 점심, 저녁 세 끼의 경관식이 포함되어 있습니다. 이는 '그린...","보호자님, 환자의 최신 의료 기록을 바탕으로 알려드리면, 오늘 아침, 점심, 저녁 ...","오늘의 식단은 다음과 같습니다:\n\n* 아침: 흰밥, 잡곡밥, 콩나물국, 해물볶음..."
3,nnnn\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nmm\n\n,"안녕하세요, 환자분 또는 보호자님. 제가 환자분이나 가족분들이 이해하기 쉽게 의료 ...","죄송하지만, 보호자의 질문이 누락되었습니다. 환자의 상태, 식사, 약물 복용, 활동...","죄송하지만, 보호자의 질문이 누락되었습니다. 가능한 한 빨리 질문을 제공해주세요."
4,뇌경색으로 1주일째 중환자실에 있는데 회복 가능할까요?,"안녕하세요,\n\n제공된 의료 문서를 보면, 환자는 뇌경색으로 인해 중환자실에서 치...",안녕하세요. 뇌경색으로 1주일째 중환자실에 계신 환자의 회복 가능성에 대해 궁금하신...,"안녕하세요,\n\n환자의 회복 가능성은 다양한 요인에 따라 달라질 수 있습니다. 그..."
...,...,...,...,...
77,고혈압에 좋은 음식이 무엇인가요?,안녕하세요. 고혈압에 좋은 음식은 다음과 같습니다:\n\n1. 칼륨이 풍부한 과일 ...,고혈압에 좋은 음식으로는 다음과 같은 것들이 있습니다.\n\n1. 칼륨이 풍부한 과...,"고혈압에 좋은 음식은 다음과 같습니다.\n\n1. 칼륨이 풍부한 식품: 바나나, 감..."
78,식사는 건강식으로 어떻게 냐오는지요?,"해당 문서에 따르면, 환자분께서 드시는 식사는 ""그린비 루션"", ""그린비 장솔루션""...","환자의 최신 의료 기록을 보면, 경관식(그린비 루션)이 아침, 점심, 저녁으로 제공...","보호자님, 저희 병원에서는 환자분들을 위해 다양한 건강식단을 제공하고 있습니다. 환..."
79,"네, 그럼 다음주에 연락 드리겟습니다","안녕하세요, 환자분. 제가 이해하기로는 의료 문서에 있는 내용을 쉽게 설명해달라는 ...",다음주에 연락 드리겠습니다.,"네, 다음주에 연락드리겠습니다."
80,간수치는 어떤가요,간수치는 LFT라는 혈액검사로 측정됩니다. 이 검사에서는 간기능과 관련된 여러 가지...,환자의 간수치는 2023년 8월 16일에 측정된 LFT(Liver Function ...,안녕하세요!


In [50]:
prompt_n1 = '''
# 시스템 설명:
- 당신은 병원 AI 지원 시스템입니다.
- 보호자 또는 환자가 의료 상태에 대해 질문할 때, 환자의 최신 의료 기록을 바탕으로 친절하고 따뜻한 답변을 제공합니다.

------

# 임무:
- 질문에 대해 친절하고 공감 어린 스타일로 답변합니다.
- 보호자나 환자가 이해할 수 있도록 전문용어를 최대한 피하고, 쉬운 용어로 설명합니다.
- 보호자의 감정을 이해하고, 위로와 격려의 말을 전합니다.
- 환자 또는 보호자의 질문에 대해 필요한 경우 추가적인 정보를 제공하고, 상세히 설명합니다.
- 개선된 대화 사례의 형식을 참고하되 질환 관련 부분은 환자의 의료 기록에 맞춰서 변경해주세요.

------

# 시작 문구 (아래 중 하나를 무작위로 선택):
- "안녕하세요, 보호자님~"
- "어머나, 보호자님~"
- "참, 보호자님~"
- "아이고, 고생 많으세요, 보호자님~"
- "정말 마음이 아프네요, 보호자님~"
- "보호자님, 힘내세요~"
- "보호자님, 정말 고생이 많으시네요~"
- "보호자님께서도 많이 지치셨을 텐데, 힘내세요~"

------

# 지시 사항:
- 2024년 7월 1일 기준으로 가장 최근 정보를 사용합니다.
- 환자의 상태, 식사, 약물 복용, 거동 상태 등의 최신 정보를 포함하여 질문에 대한 정확한 답변을 생성합니다.
- 사용한 데이터의 출처(메타데이터)를 명확히 표시합니다.
- 개선된 대화 사례의 형식을 참고하되 질환 관련 부분은 환자의 의료 기록에 맞춰서 변경해주세요.

------

# 답변 예시 형식 및 참고자료:

## 식단에 대한 질문에 대한 답변 참고자료 :
    '아침': '흰밥, 잡곡밥, 콩나물국, 해물볶음우동, 미역줄기볶음, 도라지오이초무침, 배추김치/백김치', 'calories': 612
    '점심': '흰밥, 잡곡밥, 소고기무국, 동태강정,브로컬리두부기장무침, 연근조림, 배추김치/백김치\n혈액투석식 간식 : 새우꼬치구이', 'calories': 623
    '저녁': '흰밥, 잡곡밥, 우묵국, 단호박카레, 고추장떡, 참나물샐러드, 배추김치/백김치', 'calories': 614

## 원산지 정보:
육류: 닭 : 국내산
김치류: 알타리김치 : 알타리무(국내산),고추가루(국내산) / 깍두기,석박지 : 무(국내산),고추가루(국내산)
곡류및두부류: 잡곡류 : 찹쌀,현미찹쌀,흑미찹쌀,보리,서리태(국내산) / 기장(중국산) 
생선류: 동태,코다리(러시아산) / 낙지,쭈꾸미(베트남산) / 꽃게(중국산) / 참조기(국산)/ 참치캔(가다랑어:원양산)/ 아귀(중국산)
식육가공품: 동그랑땡(돈육:국내산)/ 해물동그랑땡(오징어:외국산, 계육:국산)/ 진미채(오징어 : 페루산)

------

## 병원비 관련 질문 참고 자료: (2024년)환자군별 | 공단청구금/본인부담금/간병비 산출내역 
- 보호자가 지불해야 하는 병원비는 다음의 공단청구금의 본인부담금과 비보험비 총액(간병비와 병실료 차액 등)으로 계산됩니다.
- 환자의 정확한 정보가 없다면, 4인실과 의료중도 환자를 기준으로 병원비를 구해주세요.
- 병원비 참고자료를 활용해 병원비의 산출 과정을 설명하세요.
- 본인부담금과 비보험비 총액을 더하는 과정을 단계별로 설명하세요.
- 답변 끝에 "정확한 병원비는 병원에 문의하시면 더 자세히 알 수 있습니다."를 추가하세요.

### 1\. 공단청구금

|     |     |     |     |     |     |     |     |     |     |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 환 자 군 | 일당정액수가 | 필요인력가산 | 환자안전관리료 | 감염관리료 | 총진료비 | 식 대 | 공단청구금 | 본인부담금 | 요양급여총액 |
| 의료최고도 | 85,050 | 51,300 | 47,100 | 66,600 | 2,716,500 | 564,300 | 2,455,350 | 825,450 | 3,280,800 |
| 의료고도 | 75,510 | 51,300 | 47,100 | 66,600 | 2,430,300 | 564,300 | 2,226,390 | 768,210 | 2,994,600 |
| 의료중도 | 64,620 | 51,300 | 47,100 | 66,600 | 2,103,600 | 564,300 | 1,965,030 | 702,870 | 2,667,900 |
| 의료경도 | 62,510 | 51,300 | 47,100 | 66,600 | 2,040,300 | 564,300 | 1,914,390 | 690,210 | 2,604,600 |
| 선택입원군 | 47,000 | 51,300 | 47,100 | 66,600 | 1,575,000 | 564,300 | 1,227,150 | 912,150 | 2,139,300 |
    - 의사/간호사 1등급, 일반식/,월30일 기준

### 2\. 별도 청구: 재활, 한방, 혈액투석 등

### 3\. 비보험: 간병비 및 병실료차액

|     |     |     |     |     |     |     |
| --- | --- | --- | --- | --- | --- | --- |
| 구 분 | 1인실 | 2인실 | 3인실 | 4인실 | 5인실 | 6인실 |
| 간병비 구분 | 1:1 개인간병 | 2:1 공동간병 | 3:1 공동간병 | 4:1 공동간병 | 5:1 공동간병 | 6:1 공동간병 |
| 간병비 | 1일 약@130,000 | 1일 약@65,000 | 1일 약@40,000 | 1일 약@30,000 | 1일 약@27,000 | 1일 약@25,000 |
| 병실료차액 | 1일 약@70,000 | 1일 약@50,000 | 1일 약@30,000 | \-  | \-  | \-  |
| 기타(기저귀 등) | 사용량 | 사용량 | 사용량 | 사용량 | 사용량 | 사용량 |

## 아버님의 상태 관련 질문에 대한 답변 형식 ([]안에 내용은 환자 의료기록을 통해 채워주세요):
- "[시작 문구] 아버님께서 [질환들 나열]로 입원하셨을 때 얼마나 걱정이 크셨을까요? 정말 많은 어려움이 있으셨겠어요. 보호자님도 많이 힘드셨겠죠? 고생 많으셨습니다."
- "다행히 아버님께서 [현재 상태]을 보여주시고 있어서 조금 안심이 되실 것 같아요. [치료 방법 나열] 받고 계시니 아버님께서 빠르게 회복하시기를 진심으로 바라고 있어요."
- "[상세 내용 추가] 의료진이 최선을 다해 치료하고 있으니 아버님이 하루하루 조금씩 나아지실 거라 믿습니다."
- "보호자님께서도 이 힘든 시기를 잘 이겨내시길 진심으로 바라며, 언제든지 도움이 필요하시거나 궁금한 점이 있으시면 말씀해 주세요. 함께 힘내요!"

## 통증 여부에 대한 질문에 대한 답변 형식 ([]안에 내용은 환자 의료기록을 통해 채워주세요):
- "[시작 문구] 아버님께서 [통증 상태]으로 많이 힘드셨겠어요. 현재 [상세 통증 정보]하고 있지만, 약물 치료로 잘 조절하고 있어요. '[추가 상태 설명]' 통증의 횟수도 줄어들었다니 정말 다행입니다."
- "[추가 치료 방안 제안 및 안내]"
- "이 힘든 시간에도 아버님께서 조금씩 회복하시는 모습에 저도 함께 기쁘고 안심이 됩니다. 언제든지 필요하시면 말씀해 주세요. 보호자님도 힘내세요, 제가 항상 응원하고 있을게요!"

### 환자 의료 기록:
{context}

------

# 마무리 문구 (아래 중 하나를 무작위로 선택):
- "보호자님께서도 많이 힘드실 텐데, 마음 깊이 위로의 말씀을 전하고 싶습니다. 필요하실 때 언제든지 말씀해 주세요. 함께 이겨냅시다!"
- "이 힘든 시간 동안 아버님께서 하루하루 잘 지내시기를 진심으로 기원합니다. 필요하신 것이나 도움이 필요하시면 언제든지 말씀해 주세요. 보호자님도 힘내세요!"
- "어머님께서 조금씩 안정되시기를 저도 깊이 기원합니다. 필요하신 게 있으시면 언제든지 말씀해 주세요. 보호자님도 힘내시고, 저는 항상 응원하고 있을게요."
- "이 힘든 여정 속에서도 어머님께서 조금씩 회복되시기를 진심으로 바랍니다. 도움이 필요하시거나 이야기 나누고 싶으시면 언제든지 말씀해 주세요. 보호자님도 힘내세요!"

------

# 보호자의 질문:
{question}

------

# 중요:
- 한국어로 답변하세요.
- 개선된 대화 사례의 형식을 참고하되 질환 관련 부분은 환자의 의료 기록에 맞춰서 변경해주세요.
- []을 포함한 안의 내용은 나타나지 않도록 해주세요

# 답변:
'''

In [ ]:
rag_model('남A', prompt)

In [52]:
ans = []
for q in questions:
    a = qa_chain.invoke(q)
    ans.append(a)

qa_solar['kind'] = ans

In [54]:
qa_solar.columns

Index(['question', 'answer', '친절답변', '간단답변', 'kind'], dtype='object')

In [55]:
qa_solar.columns = ['question', '간단답변1', '간단답변2', '간단답변3', '친절답변']

In [56]:
qa_solar.to_csv('남A_final.csv',index=False)

In [17]:
llm = ChatUpstage(api_key=os.getenv("UPSTAGE_API_KEY"))


# Embeddings setup
embeddings = UpstageEmbeddings(
  api_key=os.getenv("UPSTAGE_API_KEY"),
  model="solar-embedding-1-large"
)

vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings
)

retriever = vectordb.as_retriever()

RAG_PROMPT_TEMPLATE='''
### 시스템 설명
당신은 병원 AI 지원 시스템입니다. 보호자 또는 환자가 의료 상태와 관련된 질문을 하면, 환자의 최신 의료 기록을 기반으로 적절한 답변을 제공해야 합니다.
보호자와 환자에게 알기 쉽게 친절하게 답변해주세요.
아래에 환자의 의료 기록이 제공됩니다.

### 의료 기록
{context}

### 질문
{question}

### 응답
환자의 상태, 식사, 약물 복용, 거동 상태 등을 포함하여 질문에 대한 정확한 답변을 생성하십시오.
참고한 데이터의 메타데이터를 알려주세요
'''

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the RAG chain
qa_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [18]:
query = "가장 최근 검사의 전반적인 결과가 어떻게 될까?"
print(qa_chain.invoke(query))

가장 최근 검사의 전반적인 결과는 "V/ S) BP;135 BPL;90 P;70 RESP;20 T;36\.0"으로 기록되었습니다. 이는 혈압, 맥박, 호흡, 체온을 포함한 환자의 활력 징후(Vital Signs)를 나타냅니다. 혈압은 135/90, 맥박은 70, 호흡은 20, 체온은 36도입니다. 이 정보는 2024-06-13 10:00에 기록된 것입니다.


In [19]:
print(qa_chain.invoke('말씀은 잘 하세요?'))

환자의 의료 기록에 따르면, 환자는 언어 장애가 있을 수 있습니다. 그러나 의료 기록에는 환자의 언어 상태에 대한 구체적인 정보가 없습니다. 따라서, 환자의 언어 상태를 확인하기 위해서는 의료진에게 문의하시는 것이 좋습니다.

데이터 출처: 의료 기록 (2023년 8월 16일 06:00, 2024년 2월 2일 06:00, 2024년 4월 28일 06:00)


In [20]:
print(qa_chain.invoke('밤에 잘 주무세요?'))

환자분의 의료 기록에 따르면, 최근 두 날 동안 환자분은 밤새도록 특이사항 없이 잘 주무셨다고 나와있습니다. 이 데이터는 2024년 06월 17일과 2024년 06월 16일의 간호 기록에서 확인되었습니다.

메타데이터:

* 환자: 0092106935 (남, 77세)
* 병실: 512호
* 날짜: 2024년 06월 17일, 2024년 06월 16일
* 출처: 간호 기록 (Nurse Record)


In [21]:
print(qa_chain.invoke('거동은 어떠세요?'))

환자는 와상 상태로, 스스로 일어나 앉을 수 없으며, 지지대 없이는 자세 유지가 어렵습니다.


In [22]:
print(qa_chain.invoke('약은 잘 드세요?'))

저는 환자의 의료 기록을 참고하여 답변을 생성했습니다. 해당 기록에 따르면, 환자는 아침 식후에 MD255, MD014, MD609, MD417 약물을 복용하고, 저녁 식후에는 MD417 약물을 복용합니다. 또한, 취침 전에는 MD523 약물을 복용합니다. 또한, 환자의 식사 상태는 경관식(그린비아 장솔루션)으로, 아침, 점심, 저녁에 각각 식이Y07을 복용합니다.


In [23]:
print(qa_chain.invoke('많이 아파하지는 않으신 가요?'))

제가 가진 정보에 따르면 환자는 많은 통증을 느끼지 않는 것으로 보입니다. 환자의 의료 기록에는 통증과 관련된 내용이 명시되어 있지 않습니다. 환자는 왼쪽 편마비, 대소변 조절 불가, 반복적인 행동장애 등의 증상을 겪고 있습니다. 그러나 의료 기록에는 환자가 많은 통증을 느끼고 있다는 내용이 없습니다.

제가 참고한 데이터는 환자의 의료 기록과 간호 기록으로, 환자의 상태, 식사, 약물 복용, 거동 상태 등을 포함하고 있습니다. 이 정보는 2023-11-14부터 2023-07-10까지의 기간 동안 기록된 것입니다. 의료 기록에는 환자의 진료 기록, 검사 결과, 처방된 약물, 간호 기록 등이 포함되어 있습니다. 간호 기록에는 환자의 상태 관찰, 간호 처치, 체위 변경, 욕창 관리 등에 대한 내용이 담겨 있습니다.


In [24]:
print(qa_chain.invoke('많이 아파하지는 않으신 가요?'))

환자의 기록에 따르면, 환자는 현재 좌측 편마비 상태이며, 대소변 조절이 불가능하여 기저귀를 착용하고 있습니다. 또한, 지속적인 기침과 호흡곤란이 있으며, 가슴 X-ray 검사에서 폐부종이 확인되었습니다. 또한, 환자가 발열이 있으며, 이에 대한 치료를 받고 있습니다. 이를 종합해보면, 환자는 현재 많이 아파하고 있을 가능성이 높습니다.

**참고한 데이터의 메타데이터:**

* 환자 성별: 남성
* 나이: 77세
* 병실 번호: 512호, 312호
* 날짜: 2023년 11월 14일
* 시간: 10시, 14시, 16시, 19시, 23시
* 담당 의사: FM1
* 보험 정보: 00000000000 / 명칭없음 /
* 진단: 상세불명의 뇌내출혈 (I619^00), 수면 개시 및 유지 장애[ 불면증] (G470^00), 상세불명의 위장염 및 결장염 (A099^00), 폐부종 (J81^00), 기타 및 상세불명의 원발성 고혈압 (I109^00), 합병증을 동반하지 않은 2형 당뇨병 (E119^00), 상세불명의 고지질혈증 (E785^00), 욕창궤양 및 압박부위 제2단계 (L891^00), 반코마이신 내성 (U830^00), 카바페넴계내성 (U8280^00), 기타 세균성 폐렴 (J158^00)
* 약물 복용: 인후통 (J029^06), 음식 또는 구토물에 의한 폐렴 (J690^00), 상세불명의 폐부종 (G819^00), 기타 및 상세불명의 원발성 고혈압 (I109^00), 합병증을 동반하지 않은 2형 당뇨병 (E119^00), 상세불명의 고지질혈증 (E785^00), 욕창궤양 및 압박부위 제2단계 (L891^00), 반코마이신 내성 (U830^00), 카바페넴계내성 (U8280^00), 기타 세균성 폐렴 (J158^00)
* 식이: 경관식 (그린비아 장솔루션)
* 의료 기록: 체위변경, 피부 상처 드레싱, 에어매트 적용 등.
* 의료 기록 작성자: 간호사
* 의료 기록 작성 날짜: 2023년 11월 14일
* 의료 기록 작성자 서명: (인)


In [25]:
q = '아버님은 어떠세요?'
qa_chain.invoke(q)

'아버님께서는 77세 남성이며, 312호에 입원 중이십니다. 와상환자이며, 왼쪽 편마비가 있습니다. L-tube (18Fr)를 삽입하고 있으며, 경관식을 통해 영양을 공급받고 있습니다. 대소변 조절이 불가능하여 기저귀를 착용하고 있습니다. 체위 변경 및 등 마사지를 받으며, 낙상 예방을 위한 교육이 이루어지고 있습니다. 욕창 예방을 위해 코크스 G1에 대한 처치가 이루어지고 있습니다.\n\n환자의 상태는 안정적이며, 기침과 가래 증상은 호전되고 있습니다. 담당 의사의 처방에 따라 암펙솔정(암브록솔염산염) 30mg를 복용하고 있으며, 금일 저녁부터 D/C 예정입니다. 식사는 경관식(그린비아 장솔루션)으로 400kcal/day를 섭취하고 있습니다.\n\n잠재적인 약물 상호작용:\n\n1. 리나글립틴(트라젠타정5mg)과 네비보롤(네비스톨정2.5mg)은 모두 당뇨병과 고혈압 치료에 사용됩니다. 두 약물은 일반적으로 함께 사용될 수 있지만, 드물게 저혈당 발생 위험이 증가할 수 있습니다.\n2. 아스피린(비엘피린장용정100mg)은 항응고제로 사용될 수 있으며, 다른 항응고제나 혈전용해제와 함께 사용할 경우 출혈 위험이 증가할 수 있습니다.\n3. 알프라졸람(알프람정0.25mg)은 벤조디아제핀계 약물로, 다른 중추신경계 억제제와 함께 사용할 경우 중추신경계 억제 효과가 증가할 수 있습니다.\n\n이러한 약물 상호작용 가능성을 고려하여 환자의 상태를 모니터링하고, 적절한 조치를 취해야 합니다.'